# EchoCare: Cry Classification Model Training
## Stage 2 of Two-Stage Pipeline

This notebook trains a binary classification model to detect pain cries vs hungry cries in infants.

**Dataset:**
- Baby Chillanto Dataset 

**Categories:**
- Pain
- Hungry

**Target Performance:** >70% accuracy for cry classification

**Architecture:** MobileNetV2 (lightweight for Raspberry Pi deployment) with custom classification head

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 2. Configuration

In [ ]:
# Paths
train_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/train")
val_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/validate")
test_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_classification/test")
save_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/model/cry_classification")

# Model hyperparameters
img_size = (224, 224)  # MobileNetV2 input size
batch_size = 32
epochs = 30
learning_rate = 0.0001
dropout_rate1 = 0.3
dropout_rate2 = 0.2

# Class information
class_names = ['pain', 'hungry']
num_classes = 2

## 3. Data Loading Functions

In [ ]:
def load_spectrograms(data_dir, verbose=True):
    """
    Load .npy spectrograms from directory structure.

    Args:
        data_dir: Path object or string path to directory containing pain/hungry folders
        verbose: Print loading progress
    
    Returns:
        spectrograms: numpy array of spectrograms
        labels: numpy array of labels (0=pain, 1=hungry)
    """

    spectrograms = []
    labels = []
    
    # Load hungry mel-spectrograms (label = 1)
    hungry_dir = data_dir / 'hungry'  # Find the 'hungry' subfolder
    hungry_files = list(hungry_dir.glob('*.npy')) # Get all .npy files in the 'hungry' folder
    
    for file in hungry_files: # Load each hungry file and label it as 1 (hungry = 1)
        spec = np.load(file) # load the .npy (mel spectrogram) file
        spectrograms.append(spec) # add to mel-spectrograms list
        labels.append(1) # label as hungry (1)
    
    if verbose:
        print(f"Loaded {len(hungry_files)} hungry spectrograms")
    
    # Load pain spectrograms (label = 0)
    pain_dir = data_dir / 'pain'
    pain_files = list(pain_dir.glob('*.npy'))
    
    for file in pain_files: # Load each pain file and label it as 0 (pain = 0)
        spec = np.load(file)
        spectrograms.append(spec)
        labels.append(0)
    
    if verbose:
        print(f"Loaded {len(pain_files)} pain spectrograms")
        print(f"Total samples: {len(spectrograms)}")
    
    return np.array(spectrograms), np.array(labels)

## 4. Prepare Mel-Spectrograms for MobileNetV2 Architecture

In [ ]:
def prepare_for_mobilenet(spectrograms, target_size=(224, 224)):
    """
    Prepare mel-spectrograms for MobileNetV2 input.
    
    Transforms all mel-spectrograms from (128, 32) grayscale format to (224, 224, 3) RGB format required by MobileNetV2's pre-trained weights from ImageNet.
    
    Process:
    1. Input: (128, 32) - 128 mel bands x 32 time frames
    2. Add channel: (128, 32, 1) - add dimension for grayscale
    3. Resize: (224, 224, 1) - match MobileNetV2 input size
    4. Repeat channels: (224, 224, 3) - convert to RGB by duplicating grayscale values
    
    Args:
        spectrograms: numpy array of mel-spectrograms with shape (n_samples, 128, 32)
        target_size: tuple (height, width) for resizing, default (224, 224)
    
    Returns:
        prepared: numpy array with shape (n_samples, 224, 224, 3)
    """
    prepared = []
    
    for spec in spectrograms:
        # Step 1: Add channel dimension
        # Transform from (128, 32) to (128, 32, 1)
        # This prepares the 2D mel-spectrogram for image processing
        if len(spec.shape) == 2:
            spec = np.expand_dims(spec, axis=-1)  # Expand the shape of the array. Shape: (128, 32, 1)
        
        # Step 2: Resize to MobileNetV2's expected input size
        # Transform from (128, 32, 1) to (224, 224, 1)
        # Width stretches from 32 → 224 frames (7x expansion)
        # Height expands from 128 → 224 mel bands (~1.75x expansion)
        resized = cv2.resize(spec, target_size)  # Expand the shape of the array. Shape: (224, 224, 1)
        
        # Step 3: Ensure channel dimension exists after resize
        if len(resized.shape) == 2:
            resized = np.expand_dims(resized, axis=-1)  # Shape: (224, 224, 1)
        
        # Step 4: Convert grayscale (1 channel) to RGB (3 channels)
        # Transform from (224, 224, 1) to (224, 224, 3)
        # Creates "fake RGB" by duplicating the same values across R, G, B channels
        # This allows MobileNetV2 (trained on colour images) to process the grayscale spectrograms
        if resized.shape[-1] == 1:
            resized = np.repeat(resized, 3, axis=-1)  # Final shape: (224, 224, 3)
        
        prepared.append(resized) # add to prepared list
    
    return np.array(prepared, dtype=np.float32)

## 5. Load and Visualise Cry Classification (Pain vs Hungry) Data

In [ ]:
# Load training data
print("Loading training data...")
X_train_raw, y_train = load_spectrograms(train_dir)

# Load validation data
print("\nLoading validation data...")
X_val_raw, y_val = load_spectrograms(val_dir)

# Load test data
print("\nLoading test data...")
X_test_raw, y_test = load_spectrograms(test_dir)

print(f"Training samples: {len(X_train_raw)}")
print(f"Validation samples: {len(X_val_raw)}")
print(f"Test samples: {len(X_test_raw)}")
print(f"Raw spectrogram shape: {X_train_raw[0].shape}")

In [ ]:
# Visualise sample spectrograms
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Find 4 hungry samples (where label = 1)
hungry_indices = np.where(y_train == 1)[0][60:64]

# Find 4 pain samples (where label = 0)  
pain_indices = np.where(y_train == 0)[0][60:64]

# Plot 4 hungry samples in top row
for i in range(4):
    spec = X_train_raw[hungry_indices[i]]  # Get hungry mel-spectrogram (128 mel bands × 32 time frames)
    axes[0, i].imshow(spec, aspect='auto')
    axes[0, i].set_title(f'Hungry Sample {i+1}')
    axes[0, i].axis('off')

# Plot 4 pain samples in bottom row
for i in range(4):
    spec = X_train_raw[pain_indices[i]]  # Get pain mel-spectrogram (128 mel bands × 32 time frames)
    axes[1, i].imshow(spec, aspect='auto')
    axes[1, i].set_title(f'Pain Sample {i+1}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 6. Normalising Mel-Spectrograms

In [ ]:
# Calculate normalisation parameters from training set only
mean = np.mean(X_train_raw)
std = np.std(X_train_raw)

print(f"Training set statistics:")
print(f"Mean: {mean:.2f}")
print(f"Std:  {std:.2f}")
print(f"Min:  {np.min(X_train_raw):.2f}")
print(f"Max:  {np.max(X_train_raw):.2f}")

# Apply standardisation (z-score normalisation)
X_train_norm = (X_train_raw - mean) / std
X_val_norm = (X_val_raw - mean) / std
X_test_norm = (X_test_raw - mean) / std

print("\nAfter normalisation:")
print(f"Training mean: {np.mean(X_train_norm):.2f} ") # (should be ~0)
print(f"Training std:  {np.std(X_train_norm):.2f} ") # (should be ~1)

# Show normalisation parameters for deployment on Raspberry Pi
normalisation_params = {
    "mean": float(mean),
    "std": float(std),
    "method": "standardisation"
}

print(normalisation_params)

## 7. Preparing Data for MobileNetV2

In [ ]:
# Resize and convert to 3-channel format
print("Preparing data for MobileNetV2")

X_train = prepare_for_mobilenet(X_train_norm, target_size=img_size) # prepare training set
X_val = prepare_for_mobilenet(X_val_norm, target_size=img_size) # prepare validation set
X_test = prepare_for_mobilenet(X_test_norm, target_size=img_size) # prepare test set

print(f"\nFinal shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"\n  y_train: {y_train.shape}")
print(f"  y_val:   {y_val.shape}")
print(f"  y_test:  {y_test.shape}")

## 8. Calculating Class Weights

In [ ]:
# Compute class weights to handle imbalance
# Training set has: 245 hungry sounds, 134 pain sounds

class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

print("Class weights to handle imbalance:")
for cls, weight in class_weights.items():
    print(f"{class_names[cls]}: {weight:.4f}")

print("\nThis gives more importance to the minority class (pain) during training.")

## 9. Building Cry Classification Model

In [ ]:
def build_cry_classification_model(input_shape=(224, 224, 3), dropout_rate1=0.3, dropout_rate2=0.2):
    """
    Build MobileNetV2-based binary classifier for cry classification.
    
    Args:
        input_shape: Input image shape
        dropout_rate: Dropout rate for preventing overfitting
    
    Returns:
        model: Compiled Keras model
    """
    # Load MobileNetV2 (without top classification layer)
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False, # Exclude final classification layer
        weights='imagenet'
    )
    
    # Freeze base model layers
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', name='dense_256')(x)
    x = Dropout(dropout_rate1, name='dropout_1')(x)
    x = Dense(64, activation='relu', name='dense_64')(x)
    x = Dropout(dropout_rate2, name='dropout_2')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)  # Binary classification
    
    # Create model
    model = Model(inputs=base_model.input, outputs=output, name='CryClassificationModel')
    
    return model

In [ ]:
# Build the model
model = build_cry_classification_model(
    input_shape=(img_size[0], img_size[1], 3), # MobileNetV2 expects 3-channel input
    dropout_rate1=dropout_rate1,
    dropout_rate2=dropout_rate2
)

## 10. Compiling the Model

In [ ]:
# Compile model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("Model compiled successfully")
print(f"\nOptimiser: Adam (lr={learning_rate})")
print(f"Loss: Binary Crossentropy")
print(f"Metrics: Accuracy, Precision, Recall")

## 11. Setting up Callbacks

In [ ]:
# Define callbacks
callbacks = [

    # Save best model based on validation accuracy
    ModelCheckpoint(
        filepath=str(save_dir / 'best_model_classification.keras'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    ),
    
    # Stop training when validation loss stops improving
    EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=7,
        restore_best_weights=True
    )
]

print("Callbacks configured:")
print("ModelCheckpoint")
print("EarlyStopping")

## 12. Training the Model

In [ ]:
print("Starting training...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    class_weight=class_weights,  # Handle class imbalance
    callbacks=callbacks,
    verbose=1
)

print("Training completed")

## 13. Plotting the Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].legend()

# Loss
axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Validation')
axes[0, 1].set_title('Model Loss')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].legend()

# Precision
axes[1, 0].plot(history.history['precision'], label='Train')
axes[1, 0].plot(history.history['val_precision'], label='Validation')
axes[1, 0].set_title('Model Precision')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].legend()

# Recall
axes[1, 1].plot(history.history['recall'], label='Train')
axes[1, 1].plot(history.history['val_recall'], label='Validation')
axes[1, 1].set_title('Model Recall')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 14. Loading Best Model and Evaluating on Test Set

In [ ]:
# Load the best saved model
best_model = keras.models.load_model(save_dir / 'best_model_classification.keras') 
print("Best model loaded successfully")

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = best_model.evaluate(X_test, y_test, verbose=1)

print("Results on Test Set")
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Accuracy: {test_results[1]:.4f}")
print(f"Test Precision: {test_results[2]:.4f}")
print(f"Test Recall: {test_results[3]:.4f}")

## 15. Generating Confusion Matrix

In [ ]:
# Generate predictions on test set
y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Generate and display confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Cry Classification')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

# Print confusion matrix values
print("\nConfusion Matrix:")
print(f"True Negatives (pain correctly identified): {cm[0, 0]}")
print(f"False Positives (pain wrongly as hungry): {cm[0, 1]}")
print(f"False Negatives (hungry wrongly as pain): {cm[1, 0]}")
print(f"True Positives (hungry correctly identified):  {cm[1, 1]}")

## 16. Summary

In [ ]:
print("Training Summary")

print(f"\nModel Performance:")
print(f"  Test Accuracy: {test_results[1]*100:.2f}%")
print(f"  Test Precision: {test_results[2]*100:.2f}%")
print(f"  Test Recall: {test_results[3]*100:.2f}%")